# Cutting planes enhanced by GCN models and Random actions for branching

# RANDOM ACTIONS

In [ ]:
!pip install dgl

     |████████████████████████████████| 3.0MB 8.5MB/s 


In [ ]:
!pip install pulp

     |████████████████████████████████| 40.6MB 94kB/s 


In [ ]:
import numpy as np
import pandas as pd 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch as th

from copy import deepcopy
from datetime import datetime
import random 
from collections import deque
from dgl import DGLGraph
import seaborn as sns
import matplotlib.pyplot as plt 
import networkx as nx
from sklearn.manifold import TSNE
from google.colab import files

DGL backend not selected or invalid.  Assuming PyTorch for now.
Using backend: pytorch


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
# mijn functions
import actions_v12 as act
import graph_models_v17 as gm

Valid inequalities version: 9
No hard restrictions for teeth assignments


**Champion GCN models**

In [ ]:
# Models
GCN1 = gm.GCN5_MN(4, 256, 8, 0.2)
GCN1.load_state_dict(th.load("GCN1_JUN3.pth.tar"))
GCN1.eval()

GCN2 = gm.GCN5_MX(4, 256, 8, 0.2)
GCN2.load_state_dict(th.load("GCN2_JUN3.pth.tar"))
GCN2.eval()

GCN3 = gm.GCN5_SU(4, 256, 8, 0.2)
GCN3.load_state_dict(th.load("GCN3_JUN3.pth.tar"))
GCN3.eval()

GAT2 = gm.GAT3_MX(4, 16, 8, 8, 0.2)
GAT2.load_state_dict(th.load("GAT2_JUN4.pth.tar"))
GAT2.eval()

GAT3 = gm.GAT3_SU(4, 16, 8, 8, 0.2)
GAT3.load_state_dict(th.load("GAT3_JUN4.pth.tar"))
GAT3.eval()

# Thresholds
# 0,1,2,3,4,5,7,8
tholds = [0.449, 0.343, 0.377, 0.274, 0.593, 0.336, 0.261, 0.223]

**Take actions**

In [ ]:
def pick_actions(problem, tholds, model1, model2, model3, verbose):

    # 0: subtour elimination
    # 1: blossoms (basic combs)
    # 2: advanced comb
    # 3: clique tree
    # 4: blossom + path 
    # 5: bipartition
    # 6: envelope (NOT APPLICABLE)
    # 7: crown 8 
    # 8: crown multiple 
    
    graph1 = problem.graph 
    graph1.add_edges_from(zip(graph1.nodes(), graph1.nodes()))
    graph1 = DGLGraph(graph1)

    pred1, g_emb1, n_emb1 = model1(graph1)
    pred2, g_emb2, n_emb2 = model2(graph1)
    pred3, g_emb3, n_emb3 = model3(graph1)

    counter0 = 0 
    counter1 = 0 
    counter2 = 0 
    counter3 = 0 
    counter4 = 0 
    counter5 = 0 
    counter7 = 0 
    counter8 = 0 
    cycle_cutoff = 0.99 
    teeth_cutoff = 0.1
    teeth_cutoff_path = 1
    connection_check_flag = 1 # set this as 1 in order to avoid redundant constraints 
    false_alarm = 0
    true_alarm = 0

    # 0: subtour elimination (GCN1 - GCN3)
    action0 = 1 if (pred1[0][0].detach().item() + pred3[0][0].detach().item())/2 >= tholds[0] else 0 
    if action0 == 1: # go find isolated islands and add subtours 
        counter0 = problem.subtour_elimn()
        if counter0 > 0:
          true_alarm = true_alarm + 1
          if verbose == 1:
            print('yes, there were subtours indeed:', counter0)
        else: 
          false_alarm = false_alarm + 1
          if verbose == 1:
            print('nope, there were NO subtours, wrong prediction')

    # 1: basic blossom inequalities (GCN3)
    action1 = 1 if pred3[0][1].detach().item() >= tholds[1] else 0 
    if action1 == 1:
        counter1 = problem.find_multi_blossoms(cycle_cutoff, teeth_cutoff)
        if counter1 > 0:
          true_alarm = true_alarm + 1
          if verbose == 1:
            print('yes, there were blossoms indeed:', counter1)
        else: 
          false_alarm = false_alarm + 1
          if verbose == 1:
            print('nope, there were NO blossoms, wrong prediction')

    # 2: advanced comb inequalities (GCN3)
    action2 = 1 if pred3[0][2].detach().item() >= tholds[2] else 0 
    if action2 == 1: 
        counter2 = problem.find_adv_combs(cycle_cutoff, teeth_cutoff)
        if counter2 > 0:
          true_alarm = true_alarm + 1
          if verbose == 1:
            print('yes, there were advanced combs indeed:', counter2)
        else: 
          false_alarm = false_alarm + 1
          if verbose == 1:
            print('nope, there were NO advanced combs, wrong prediction')

    # 3: clique-tree inequality (GCN 1-2-3)
    action3 = 1 if (pred1[0][3].detach().item() + pred2[0][3].detach().item() + pred3[0][3].detach().item())/3 >= tholds[3] else 0 
    if action3 == 1: 
        counter3 = problem.find_clique_tree_2(cycle_cutoff, teeth_cutoff) 
        if counter3 > 0:
          counter3 = 1
          true_alarm = true_alarm + 1
          if verbose == 1:
            print('yes, there was a clique-tree indeed')
        else: 
          false_alarm = false_alarm + 1
          if verbose == 1:
            print('nope, there was NO clique-tree, wrong prediction')
        
    # 5: bipartiton inequality (GCN1 - GCN2)
    action5 = 1 if (pred1[0][5].detach().item() + pred2[0][5].detach().item())/2 >= tholds[5] else 0 
    if action5 == 1:
        counter5 = problem.find_bipartition(cycle_cutoff, teeth_cutoff) 
        if counter5 > 0:
          counter5 = 1
          true_alarm = true_alarm + 1
          if verbose == 1:
            print('yes, there was a bipartiton indeed')
        else: 
          false_alarm = false_alarm + 1
          if verbose == 1:
            print('nope, there was NO bipartition, wrong prediction')

    # 4: blossom and path inequality (GCN1 - GCN3)
    action4 = 1 if (pred1[0][4].detach().item() + pred3[0][4].detach().item())/2 >= tholds[4] else 0 
    if action4 == 1:
        counter4, xd1 = problem.find_blossom_n_path(cycle_cutoff, teeth_cutoff, teeth_cutoff_path) 
        if counter4 > 1:
          true_alarm = true_alarm + 1
          if verbose == 1:
            print('yes, there were blossom and following path indeed')
        elif counter4 == 1:
          true_alarm = true_alarm + 1
          if verbose == 1:
            print('hmm, there was only a blossom here')
        else: 
          false_alarm = false_alarm + 1
          if verbose == 1:
            print('nope, there were NO blossom and path, wrong prediction')

    # 7: crown with 8 subsets (GCN1 - GCN3)
    action7 = 1 if (pred1[0][6].detach().item() + pred3[0][6].detach().item())/2 >= tholds[6] else 0 
    if action7 == 1:
        counter7 = problem.find_crown_8(connection_check_flag)
        if counter7 > 1:
          true_alarm = true_alarm + 1
          counter7 = 1
          if verbose == 1:
            print('yes, there was a crown with 8 subsets indeed')
    
    # 8: crown with N subsets (GCN1 - GCN3)
    action8 = 1 if (pred1[0][7].detach().item() + pred3[0][7].detach().item())/2 >=  tholds[7] else 0 
    if action8 == 1: # go find advanced combs
        counter8 = problem.find_crown_more(connection_check_flag)
        if counter8 > 1:
          true_alarm = true_alarm + 1
          counter8 = 1
          if verbose == 1:
            print('yes, there was a crown with N subsets indeed')

    count_total = counter0 + counter1 + counter2 + counter3 + counter4 + counter5 + counter7 + counter8

    return problem, count_total, true_alarm, false_alarm

In [ ]:
def plot_graph(problem1):
    
    # load nodes
    graph = nx.Graph()
    graph.add_nodes_from(problem1.node_list)
    
    if len(problem1.X_soln) > 0:
      
        for k in range(len(problem1.X_soln)):
            graph.add_edge(problem1.X_soln.loc[k][0], problem1.X_soln.loc[k][1], capacity=1.0)
         
        problem1.X_soln['color'] = problem1.X_soln.apply(lambda x: 'b' if x.x_value >= 0.99  else 'r', axis=1)
        colors = problem1.X_soln['color'].to_list()
    
        #set coordinates and plot
        nx.set_node_attributes(graph, problem1.coordinates_dict, 'pos')    
        nx.draw(graph, nx.get_node_attributes(graph, 'pos'), with_labels = True, edge_color=colors)
        #plt.savefig("simple_path.png") 
        plt.show() 

**Functions for reward, branch-check and state**

In [ ]:
def check_branch(problem):

  soln = problem.X_soln 
  fractional = soln[soln.x_value < 1]
  if len(fractional) > 0: 
    branch_flag = 1
  else:
    branch_flag = 0

  return branch_flag

# define state
def define_state_for_random(problem):
  
  soln = problem.X_soln 
  fractional = soln[soln.x_value < 1]
  
  if len(fractional) > 0:
    node = fractional.iloc[0]['origin']
    connection = fractional.iloc[0]['destination']
    #regret = list(fractional[fractional['origin'] == node]['destination'].append(fractional[fractional['destination'] == node]['origin']))
    #regret = [item for item in regret if item != connection]
    #regret = regret[0]
  else:
    node = 0 #assign redundant values
    connection = 1 

  return node, connection

**Train/Test instances**

**TSP list**

In [ ]:
train_tsp_list = [
(38,'TestTSP0_3.csv'),
(39,'TestTSP0_4.csv'),
(40,'TestTSP0_5.csv'),
(41,'TestTSP0_6.csv'),
(42,'TestTSP0_7.csv'),
(43,'TestTSP0_8.csv'),
(44,'TestTSP0_9.csv'),
(46,'TestTSP0_11.csv'),
(47,'TestTSP0_12.csv'),
(48,'TestTSP0_13.csv'),
(49,'TestTSP0_14.csv'),
(87,'TestTSP1_2.csv'),
(88,'TestTSP1_3.csv'),
(89,'TestTSP1_4.csv'),
(90,'TestTSP1_5.csv'),
(91,'TestTSP1_6.csv'),
(93,'TestTSP1_8.csv'),
(94,'TestTSP1_9.csv'),
(96,'TestTSP1_11.csv'),
(97,'TestTSP1_12.csv'),
(98,'TestTSP1_13.csv'),
(99,'TestTSP1_14.csv'),
(136,'TestTSP2_1.csv'),
(142,'TestTSP2_7.csv'),
(144,'TestTSP2_9.csv'),
(145,'TestTSP2_10.csv'),
(147,'TestTSP2_12.csv'),
(148,'TestTSP2_13.csv'),
(149,'TestTSP2_14.csv'),
(183,'TestTSP3_14.csv'),
(185,'TestTSP3_0.csv'),
(195,'TestTSP3_10.csv'),
(233,'TestTSP4_0.csv'),
(234,'TestTSP4_1.csv'),
(235,'TestTSP4_2.csv'),
(236,'TestTSP4_3.csv'),
(237,'TestTSP4_4.csv'),
(238,'TestTSP4_5.csv'),
(239,'TestTSP4_6.csv'),
(240,'TestTSP4_7.csv'),
(241,'TestTSP4_8.csv'),
(242,'TestTSP4_9.csv'),
(243,'TestTSP4_10.csv'),
(244,'TestTSP4_11.csv'),
(245,'TestTSP4_12.csv'),
(246,'TestTSP4_13.csv'),
(247,'TestTSP4_14.csv'),
(283,'TestTSP5_0.csv'),
(284,'TestTSP5_1.csv'),
(285,'TestTSP5_2.csv'),
(286,'TestTSP5_3.csv'),
(287,'TestTSP5_4.csv'),
(288,'TestTSP5_5.csv'),
(289,'TestTSP5_6.csv'),
(292,'TestTSP5_9.csv'),
(294,'TestTSP5_11.csv'),
(297,'TestTSP5_14.csv'),
(335,'TestTSP6_2.csv'),
(336,'TestTSP6_3.csv'),
(346,'TestTSP6_13.csv'),
(383,'TestTSP7_0.csv'),
(384,'TestTSP7_1.csv'),
(385,'TestTSP7_2.csv'),
(386,'TestTSP7_3.csv'),
(387,'TestTSP7_4.csv'),
(388,'TestTSP7_5.csv'),
(389,'TestTSP7_6.csv'),
(390,'TestTSP7_7.csv'),
(391,'TestTSP7_8.csv'),
(392,'TestTSP7_9.csv'),
(393,'TestTSP7_10.csv'),
(394,'TestTSP7_11.csv'),
(395,'TestTSP7_12.csv'),
(396,'TestTSP7_13.csv'),
(397,'TestTSP7_14.csv'),
(433,'TestTSP8_0.csv'),
(438,'TestTSP8_5.csv'),
(439,'TestTSP8_6.csv'),
(442,'TestTSP8_9.csv'),
(445,'TestTSP8_12.csv'),
(447,'TestTSP8_14.csv'),
(485,'TestTSP9_2.csv'),
(486,'TestTSP9_3.csv'),
(488,'TestTSP9_5.csv'),
(489,'TestTSP9_6.csv'),
(490,'TestTSP9_7.csv'),
(492,'TestTSP9_9.csv'),
(494,'TestTSP9_11.csv'),
(495,'TestTSP9_12.csv'),
(497,'TestTSP9_14.csv'),
(533,'TestTSP10_0.csv'),
(534,'TestTSP10_1.csv'),
(535,'TestTSP10_2.csv'),
(536,'TestTSP10_3.csv'),
(537,'TestTSP10_4.csv'),
(540,'TestTSP10_7.csv'),
(541,'TestTSP10_8.csv'),
(542,'TestTSP10_9.csv'),
(545,'TestTSP10_12.csv'),
(546,'TestTSP10_13.csv'),
(547,'TestTSP10_14.csv'),
(583,'TestTSP11_0.csv'),
(584,'TestTSP11_1.csv'),
(586,'TestTSP11_3.csv'),
(587,'TestTSP11_4.csv'),
(588,'TestTSP11_5.csv'),
(590,'TestTSP11_7.csv'),
(591,'TestTSP11_8.csv'),
(592,'TestTSP11_9.csv'),
(631,'TestTSP12_0.csv'),
(632,'TestTSP12_1.csv'),
(634,'TestTSP12_3.csv'),
(635,'TestTSP12_4.csv'),
(636,'TestTSP12_5.csv'),
(637,'TestTSP12_6.csv'),
(639,'TestTSP12_8.csv'),
(641,'TestTSP12_10.csv'),
(645,'TestTSP12_14.csv'),
(681,'TestTSP13_0.csv'),
(684,'TestTSP13_3.csv'),
(685,'TestTSP13_4.csv'),
(687,'TestTSP13_6.csv'),
(689,'TestTSP13_8.csv'),
(692,'TestTSP13_11.csv'),
(693,'TestTSP13_12.csv'),
(694,'TestTSP13_13.csv'),
(734,'TestTSP14_3.csv'),
(742,'TestTSP14_11.csv'),
(780,'TestTSP15_0.csv'),
(781,'TestTSP15_1.csv'),
(782,'TestTSP15_2.csv'),
(783,'TestTSP15_3.csv'),
(785,'TestTSP15_5.csv'),
(788,'TestTSP15_8.csv'),
(791,'TestTSP15_11.csv'),
(792,'TestTSP15_12.csv'),
(793,'TestTSP15_13.csv'),
(794,'TestTSP15_14.csv'),
(831,'TestTSP16_1.csv'),
(835,'TestTSP16_5.csv'),
(836,'TestTSP16_6.csv'),
(837,'TestTSP16_7.csv'),
(838,'TestTSP16_8.csv'),
(839,'TestTSP16_9.csv'),
(841,'TestTSP16_11.csv'),
(842,'TestTSP16_12.csv'),
(844,'TestTSP16_14.csv'),
(880,'TestTSP17_0.csv'),
(881,'TestTSP17_1.csv'),
(883,'TestTSP17_3.csv'),
(888,'TestTSP17_8.csv'),
(890,'TestTSP17_10.csv'),
(891,'TestTSP17_11.csv'),
(892,'TestTSP17_12.csv'),
(931,'TestTSP18_1.csv'),
(932,'TestTSP18_2.csv'),
(935,'TestTSP18_5.csv'),
(936,'TestTSP18_6.csv'),
(938,'TestTSP18_8.csv'),
(939,'TestTSP18_9.csv'),
(940,'TestTSP18_10.csv'),
(942,'TestTSP18_12.csv'),
(943,'TestTSP18_13.csv'),
(944,'TestTSP18_14.csv')
]

In [ ]:
print('number of instances:', len(train_tsp_list))

number of instances: 164


# Cutting Planes and random actions together

**Parameters**

In [ ]:
action_size = 2 
input_size = 128 * 2 + 256*2

keep_instances = []
keep_solutions = []
keep_initial_obj = []
keep_branch_flag = []

cp_random_train = pd.DataFrame()

# Solve Test instances

In [ ]:
for k in range(95, len(train_tsp_list)): 

  tsp_id = train_tsp_list[k][0]
  data_name = train_tsp_list[k][1]
  problem1 = act.init_prob(tsp_id, data_name, 0, 'continuous') 
  initial_objective = problem1.objective_val
  
  # show the initial solution
  print('initial tour')
  # plot_graph(problem1)
  constraint_count = 0
  branch_count = 0 
  action_sum = 0

  true_alarm_count = 0
  false_alarm_count = 0
  con_limit = 500
  change_flag = 1
  break_flag = 0 

  start_time = datetime.now()
  print("iteration:", k)
  print("running for tsp:", tsp_id, "start time:", start_time)

  while (constraint_count + branch_count) <= con_limit and problem1.complete_flag == 0:

    if break_flag == 1:
      break # while loop
  
    if change_flag == 0 or (constraint_count + branch_count) > con_limit/5: # keep branching either after some time as well
      # Do branching if it's possible 
      branch_flag = check_branch(problem1)
      
      if branch_flag == 1: 
        branch_count = branch_count + 1
        node, connection = define_state_for_random(problem1)
        action = random.randint(0, 1) # RANDOM ACTION HERE
        if action == 0:
          action = 3
        else:
          action = 1 
        action_sum = action_sum + action
      
        # take the action    
        problem1 = act.add_branch(problem1, node, connection, connection, action)
        problem1.solve_lp_relax()
        problem1.graph = problem1.create_graph()
        problem1.check_if_complete()

      elif branch_flag == 0 and change_flag == 0:
        break_flag = 1 # no more rooms to search through 
   
    # Also keep adding valid inequalities
    old_solution = problem1.X_soln[['origin', 'destination', 'x_value']]
    problem1, count_total, true_alarm, false_alarm = pick_actions(problem1, tholds, GCN1, GCN2, GCN3, 0)

    problem1.solve_lp_relax() 
    new_solution = problem1.X_soln[['origin', 'destination', 'x_value']]
    change_flag = 1 - int(old_solution.equals(new_solution))
    constraint_count = constraint_count + count_total    

    if change_flag == 1:
      true_alarm_count = true_alarm_count + true_alarm
      false_alarm_count = false_alarm_count + false_alarm
      problem1.graph = problem1.create_graph()
      problem1.check_if_complete()
    
    elif change_flag == 0:
      false_alarm_count = false_alarm_count + constraint_count 

  end_time = datetime.now()
  print('final tour')
  # plot_graph(problem1)
  problem1.check_if_complete()
  if problem1.complete_flag == 0:
    print('tour is NOT complete, branching needed!')
  else: 
    print('tour is complete!')

  obj1 = {'tsp_id': tsp_id, 
            'initial_objective': initial_objective, 
            'final_objective' : problem1.objective_val, 
            'start_time': start_time, 
            'end_time': end_time, 
            'complete_flag':problem1.complete_flag, 
            'n_of_constraint': constraint_count, 
            'n_of_branch': branch_count,
            'action_sum':action_sum,
            'true_alarm': true_alarm_count, 
            'false_alarm': false_alarm_count,
            'con_limit': con_limit
          }
  cp_random_train = cp_random_train.append(pd.DataFrame(obj1, index=[0]))

# download the output
cp_random_train.to_csv('CP_random_test_perfo.csv')
files.download('CP_random_test_perfo.csv') 

initial tour
iteration: 95
running for tsp: 540 start time: 2020-06-19 18:52:44.010925
final tour
tour is complete!
initial tour
iteration: 96
running for tsp: 541 start time: 2020-06-19 18:54:54.108063
final tour
tour is complete!
initial tour
iteration: 97
running for tsp: 542 start time: 2020-06-19 18:59:15.611407
final tour
tour is NOT complete, branching needed!
initial tour
iteration: 98
running for tsp: 545 start time: 2020-06-19 19:13:34.063429
final tour
tour is complete!
initial tour
iteration: 99
running for tsp: 546 start time: 2020-06-19 19:19:21.781954
final tour
tour is complete!
initial tour
iteration: 100
running for tsp: 547 start time: 2020-06-19 19:22:55.827753
final tour
tour is complete!
initial tour
iteration: 101
running for tsp: 583 start time: 2020-06-19 19:27:13.189375
final tour
tour is complete!
initial tour
iteration: 102
running for tsp: 584 start time: 2020-06-19 19:28:42.624862
final tour
tour is complete!
initial tour
iteration: 103
running for tsp: 58

In [ ]:
# download the output
cp_random_train.to_csv('CP_random_test_perfo.csv')
files.download('CP_random_test_perfo.csv') 

NameError: ignored